In [1]:
import os
import time

import torch
import numpy as np

from utilities import *
from metrics import *
from pneumonia_dataset import load_data
from unet import LeakyUNET
from loss import BCEWithLogitLoss2D
from experiment import *

In [2]:
EXP_NAME = 'UNET_SingleView_2xDownsample'
timestamp = time.strftime('%m%d-%H%M%S', time.localtime())
output_dir = os.path.join('./output/', EXP_NAME, timestamp)
os.makedirs(output_dir)

debug = True
original_dim = 1024

# TODO: change rescale_factor or batch_size
rescale_factor = 2
batch_size = 6
validation_prop = 0.1

data_dir = './data'
train_csv_path = os.path.join(data_dir, 'train.csv')
test_csv_path = os.path.join(data_dir, 'test.csv')

train_images_dir = os.path.join(data_dir, 'stage_1_train_images/')
test_images_dir = os.path.join(data_dir, 'stage_1_test_images/')

train_df, train_loader, dev_pids, dev_loader, dev_dataset_for_predict, dev_loader_for_predict, test_loader, test_df, test_pids, boxes_by_pid_dict, min_box_area = load_data(train_csv_path, test_csv_path, train_images_dir, test_images_dir, batch_size, validation_prop, rescale_factor)
min_box_area = int(round(min_box_area / float(rescale_factor**2)))

In [3]:
model = LeakyUNET().cuda()

loss_fn = BCEWithLogitLoss2D().cuda()

init_learning_rate = 0.5

num_epochs = 1 if debug else 10
num_train_steps = 5 if debug else len(train_loader)
num_dev_steps = 5 if debug else len(dev_loader)

img_dim = int(round(original_dim / rescale_factor))

print("Training for {} epochs".format(num_epochs))
histories, best_models = train_and_evaluate(model, train_loader, dev_loader, init_learning_rate,
                                          loss_fn, num_epochs, num_train_steps, num_dev_steps,
                                          boxes_by_pid_dict, rescale_factor, img_dim, output_dir, min_box_area=min_box_area)

print('- Predicting with best PRECISION model')
best_precision_model = best_models['best precision model']
torch.save(best_precision_model, os.path.join(output_dir, 'best_precision_model.pt'))
dev_predictions = predict(best_precision_model, dev_loader_for_predict)
test_predictions = predict(best_precision_model, test_loader)

# box_thresh = best_box_thresh_from_dev_predictions(dev_predictions, dev_dataset_for_predict, rescale_factor, boxes_by_pid_dict)
box_thresh = 0.2
save_predictions_to_csv(test_df, test_pids, test_predictions, box_thresh, rescale_factor, output_dir, 'best_precision')

print('- Predicting with best LOSS model')
best_loss_model = best_models['best loss model']
torch.save(best_loss_model, os.path.join(output_dir, 'best_loss_model.pt'))
dev_predictions = predict(best_loss_model, dev_loader_for_predict)
test_predictions = predict(best_loss_model, test_loader)

# box_thresh = best_box_thresh_from_dev_predictions(dev_predictions, dev_dataset_for_predict, rescale_factor, boxes_by_pid_dict)
box_thresh = 0.2
save_predictions_to_csv(test_df, test_pids, test_predictions, box_thresh, rescale_factor, output_dir, 'best_loss')

print('Yay!')

Training for 1 epochs
Epoch 1/1. Learning rate = 0.500


/nfs01/shared_software/anaconda3/envs/gia/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


--- Train batch 1 / 5: batch loss = 0.8216984 ; average loss = 0.8216984 ; batch precision = 0.0000000 ; 
    46.92 seconds


/home/ghn8/pneumonia-detection/metrics.py:162: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.asarray(batch_precisions))


--- Train batch 6 / 5: batch loss = 0.6684821 ; average loss = 0.7014893 ; batch precision = 00nan ; 
    17.58 seconds
- Train epoch metrics: 
--- Validation batch 1 / 5


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1525812548180/work/aten/src/THC/generic/THCStorage.cu:58